<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Stock_Price_Prediction_with_LSTM_and_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries

!pip install pandas_ta
!pip install yfinance
!pip install optuna

import optuna
from sklearn.model_selection import TimeSeriesSplit

import datetime
import yfinance as yf
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas_ta as ta

# Gather Data
ticker = "AAPL"  # Apple Inc. stock as an example, can replace with other stock tickers
start_date = "2010-01-01"
end_date = datetime.datetime.now().strftime("%Y-%m-%d")
df = yf.download(ticker, start=start_date, end=end_date, interval="1d")

# ... (rest of the code remains the same)

# Calculate technical indicators
df["RSI"] = ta.rsi(df["Close"])
df[["MACD_12_26", "MACD_12_26_Signal", "MACD_12_26_Hist"]] = ta.macd(df["Close"])
df["ATR"] = ta.atr(df["High"], df["Low"], df["Close"])

# Drop NaN values
df.dropna(inplace=True)

# Preprocess Data
df_scaler = MinMaxScaler()
input_features = ["Close", "RSI", "MACD_12_26", "MACD_12_26_Signal", "MACD_12_26_Hist", "ATR"]
df[input_features] = df_scaler.fit_transform(df[input_features])

close_scaler = MinMaxScaler()
df['Close'] = close_scaler.fit_transform(df[['Close']])

# Reset index
df.reset_index(drop=True, inplace=True)

# Split Data
train_size = int(len(df) * 0.8)
train_data = df[:train_size]
test_data = df[train_size:]

# Create sequences
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length - 1):
        sequence = data[i : i + seq_length].values
        label = data["Close"].iloc[i + seq_length]
        sequences.append(sequence)
        labels.append(label)
    return np.array(sequences), np.array(labels)

seq_length = 60
train_x, train_y = create_sequences(train_data[input_features], seq_length)
test_x, test_y = create_sequences(test_data[input_features], seq_length)

# Define the objective function for Optuna
def objective(trial):

    # Hyperparameter search space
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    lstm_units = trial.suggest_categorical("lstm_units", [30, 50, 80])
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)

    # Model definition
    model = Sequential()
    model.add(LSTM(units=lstm_units, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=lstm_units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mean_squared_error")

    # TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)

    for train_index, val_index in tscv.split(train_x):
        train_x_cv, val_x_cv = train_x[train_index], train_x[val_index]
        train_y_cv, val_y_cv = train_y[train_index], train_y[val_index]

        model.fit(train_x_cv, train_y_cv, epochs=10, batch_size=batch_size, validation_data=(val_x_cv, val_y_cv), verbose=0)

    val_loss = model.evaluate(val_x_cv, val_y_cv, verbose=0)
    return val_loss

# Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

print("Best hyperparameters: ", study.best_params)

# Training the model with best hyperparameters on full data
best_params = study.best_params
model = Sequential()
model.add(LSTM(units=best_params["lstm_units"], return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(Dropout(best_params["dropout_rate"]))
model.add(LSTM(units=best_params["lstm_units"]))
model.add(Dropout(best_params["dropout_rate"]))
model.add(Dense(units=1))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=best_params["learning_rate"]), loss="mean_squared_error")
model.fit(train_x, train_y, epochs=10, batch_size=best_params["batch_size"])

# ... (Rest of the code remains the same, for predictions, evaluations, etc.)


# Train Model
model.fit(train_x, train_y, epochs=10, batch_size=32)

# Make Predictions
predictions = model.predict(test_x)
predictions = close_scaler.inverse_transform(predictions)

# Evaluate Model
mse = mean_squared_error(predictions, close_scaler.inverse_transform(test_y.reshape(-1, 1)))
mae = mean_absolute_error(predictions, close_scaler.inverse_transform(test_y.reshape(-1, 1)))
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)


print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

# Save Model
model.save("forex_trading_bot_model.keras")

# Make a prediction for the next day
next_day_sequence = np.expand_dims(test_x[-1], axis=0)
next_day_prediction = model.predict(next_day_sequence)
next_day_price = close_scaler.inverse_transform(next_day_prediction)[0][0]

# Print the predicted price for the next day
print("Predicted price for the next day:", next_day_price)

# Determine the action based on the predicted price
if next_day_price > df["Close"].iloc[-1]:
    print("Action: Buy")
elif next_day_price < df["Close"].iloc[-1]:
    print("Action: Sell")
else:
    print("Action: Hold")

# Create a baseline model that predicts the last known price as the next day's price
baseline_predictions = np.roll(test_x[:,-1,0], -1)  # Shift the last known prices one step forward
baseline_mse = mean_squared_error(test_y[:-1], baseline_predictions[:-1])
baseline_mae = mean_absolute_error(test_y[:-1], baseline_predictions[:-1])

# Calculate the Mean Squared Error (MSE) and Mean Absolute Error (MAE) for the LSTM model
lstm_mse = mean_squared_error(test_y, predictions)
lstm_mae = mean_absolute_error(test_y, predictions)

# Print the comparison results
print("\nBaseline Model:")
print("MSE:", baseline_mse)
print("MAE:", baseline_mae)

print("\nLSTM Model:")
print("MSE:", lstm_mse)
print("MAE:", lstm_mae)

# Compare the LSTM model's performance with the baseline model's performance
if lstm_mse < baseline_mse and lstm_mae < baseline_mae:
    print("\nConclusion: The LSTM model's performance is better than the baseline model. Consider using LSTM predictions for trading forex.")
    if next_day_price > df["Close"].iloc[-1]:
        print("Action: Buy")
    elif next_day_price < df["Close"].iloc[-1]:
        print("Action: Sell")
    else:
        print("Action: Hold")
elif lstm_mse > baseline_mse and lstm_mae > baseline_mae:
    print("\nConclusion: The baseline model's performance is better than the LSTM model. Caution advised.")
else:
    print("\nConclusion: The LSTM model's performance is similar to the baseline model. Use additional analysis for decision-making.")